In [162]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [163]:
# Global Variables

ratedCap = 8 #MW
costPerMW = 0.9 #£1.1mil per MW
initElecPrice = 55 #£/mwh 
capFactor = 0.3 #Output is 30% of RC
availability = 0.95 #Assume turbines are on 95% of the time
numberOfTurbines=100

inflationRate = 1.05
riskAdjRate = 0.07 #risk-adjusted discount rate
riskFreeRate = 0.03

assetLife = 20
gearBoxLife = 10



impCostGearbox = 100 # millions - Set very high for single abandon case
abdValue = 500 #Set to 0 for single gearbox replacement case

In [164]:
# Legacy Function Definitions

def findSetupCost(ratedCap, numberOfTurbines, costPerMW):
    setupCost = numberOfTurbines * costPerMW * ratedCap
    return setupCost

def netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability):
    grossYield = numberOfTurbines * ratedCap * capFactor * 8760 #8760 hours in a year
    netYield = availability * grossYield
    return netYield

#assetLife, initElecPrice, inflationRate,riskAdjRate,numberOfTurbines
def cashFlowCalculation(*args):
    discountedCashFlows = np.zeros(assetLife)

    
    for i in range(0,assetLife):
        netYield = netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability)
        discountedCashFlows[i] = pow(10,-6)* 0.7 * np.random.normal(initElecPrice,3) * pow(inflationRate,i) * netYield / pow((1 + riskAdjRate), i)
        
    return discountedCashFlows


#assetLife, initElecPrice, inflationRate,riskAdjRate,numberOfTurbines
def baseCashFlowCalculation(*args):
    discountedBaseCashFlows = np.zeros(assetLife)

    
    
    for i in range(0,assetLife):
        netYield = netYieldCalculation(numberOfTurbines, ratedCap, capFactor, availability)
        discountedBaseCashFlows[i] = pow(10,-6)* 0.7 * initElecPrice * pow(inflationRate,i) * netYield / pow((1 + riskAdjRate), i)
        
    return discountedBaseCashFlows

In [165]:
# Simulation Environment Functions

def TurbineDeathSimulation(scale, loc):
    """Takes the baseCashFlows and simulates how turbine failures would change overall project npv"""
    
    discountedBaseCashFlows = baseCashFlowCalculation()
    x= np.floor(np.random.normal(loc = loc , scale = scale , size = numberOfTurbines)).astype('int')
    unique, counts = np.unique(x, return_counts=True)
    y = np.asarray((unique, 1 - np.cumsum(counts)/numberOfTurbines)).T

    if np.any(y<0):
        y = np.asarray([ [ i[0]-min(y[:,0]) , i[1] ] for i in y ])
        
        if np.any(y>assetLife-1):
            y = y[y[:,0] < assetLife]
            y[-1][1] = 0

    elif np.any(y>assetLife-1):
        y = np.asarray([ [ i[0] - ( max(y[:,0]) - (assetLife-1) ) , i[1] ] for i in y ]) 

        if np.any(y<0):
            startValue = y[0][1]
            y = y[y[:,0] > -1]
            y[0][1] = startValue
            
            
    
    probArray = np.ones(assetLife)
    for count,i in enumerate(y):
        probArray[i[0].astype('int')] = i[1]
    probArray = [0 if count>y[-1][0] else i for count,i in enumerate(probArray)]    
    
    output = discountedBaseCashFlows * probArray
    
    return output[output > 0]
    

    
    
def SigmaCalculator(testFunction,arguments):
    """Takes the price fluctuated cash flows and calculates the mean of the standard deviation of the
    logrithemic returns ,from it"""
    
    sigma_array = []
    for _ in range(0,10000):
        
        discountedCashFlows = testFunction(arguments)
        logrithmicReturns = np.zeros(len(discountedCashFlows) - 1)
        
        for g in enumerate(discountedCashFlows):
            if g[0]<len(discountedCashFlows) - 1:
                logrithmicReturns[g[0]] = np.log( discountedCashFlows[g[0]+1]/discountedCashFlows[g[0]] )

        sigma_array.append(np.std(logrithmicReturns))

    sigma = np.mean(sigma_array)
    return sigma
       
  

In [167]:
# Option Valuation Functions
def GearboxVal(tree , steps , p , delta , assetLife , impCostGearbox):
    """ Gives us the npv of the gear box replacement option applied """   

    count = 0
    optionTree = np.copy(tree)

    for i in range(0,steps):
            gain = ( pow(inflationRate,assetLife+gearBoxLife) - 1 ) / ( pow(inflationRate,assetLife) - 1 )
            case0 = (optionTree[i,steps-1] * gain) - impCostGearbox
            optionTree[i,steps-1] = max(optionTree[i,steps-1] ,case0 )

    for i in range(steps-1, 0 ,-1):
        for j in range(0,steps-1):

            case1 = ( p*optionTree[j , i] + (1-p)*optionTree[j+1 , i] ) * np.exp(-1* riskFreeRate * delta)

            extraYears = max(np.floor(delta*i) + gearBoxLife - assetLife , 0) 
            gain = ( pow(inflationRate,assetLife+extraYears) - 1 ) / ( pow(inflationRate,assetLife) - 1 ) 

            case2 = (optionTree[j , i-1] * gain) - impCostGearbox

            optionTree[j , i-1] = max(case1 , case2)    



    return optionTree




def AbandonmentVal(optionTree , steps , p , delta , abdValue):
    """ Gives us the npv of the Abandonment option applied """

    option_values = np.copy(optionTree)

    for i in range(0,steps):
        option_values[i , steps-1] = max( option_values[i , steps-1] , abdValue )

    for i in range(steps-1, 0 ,-1):
        for j in range(0,steps-1):
            case1 = ( p*option_values[j , i] + (1-p)*option_values[j+1 , i] ) * np.exp(-1* riskFreeRate * delta)
            case2 = (option_values[j , i -1] / option_values[j , i -1] ) * abdValue

            option_values[j , i -1 ] = max(case1 , case2)


    return option_values    
        

                                      
                                      
                                      
def InitCalc(cashFlows ,steps , impCostGearbox , abdValue):
    """ Brings together all the other option calculations """
    sigma2 = 0.085
    S0 = sum(cashFlows)
    assetLife = len(cashFlows)
    delta = assetLife / steps
    u = np.exp(sigma2 * np.sqrt(delta)) 
    d = 1/u
    p = ( np.exp(riskFreeRate * delta) - d ) / (u - d)
    tree = np.zeros((steps, steps))
    

    #Tree Initial Seeding  
    for i in range(0,steps):
        tree[0 , i] = S0 * pow(u,i)  


    #Tree Propagator     
    for i in range(0,steps-1):
        if i < steps-1:
            for j in range(0,steps):
                if j < steps-1:
                    tree[i+1 , j+1] = tree[i,j] * d
    tree[tree == 0] = None

    
    
    
    optionTree = GearboxVal(tree , steps , p , delta , assetLife , impCostGearbox) 
    AoptionTree = AbandonmentVal(optionTree , steps , p , delta , abdValue)
    
    return tree , optionTree , AoptionTree




In [170]:
# Testing single failure scenario trees

#sigma = SigmaCalculator(cashFlowCalculation,())
tree,optionTree,AoptionTree = InitCalc(TurbineDeathSimulation(scale = 1,loc = 10) ,steps = 31 , impCostGearbox = impCostGearbox, abdValue = abdValue)
#optionTree[0,0] - tree[0,0] # For option value
#pd.DataFrame(tree) # Base tree depending on failure scenario
#pd.DataFrame(optionTree) # Base tree depending on failure scenario , with gearbox replacement option applied
#pd.DataFrame(AoptionTree) # Base tree depending on failure scenario , with gearbox replacement + Abandonment option applied

In [171]:
European_option_value = AoptionTree[0,0]-tree[0,0]
setupCost = findSetupCost(ratedCap, numberOfTurbines, costPerMW)
BAUNPV = tree[0,0]-setupCost
European_project_value = European_option_value - setupCost

print("Setup Cost = £%.2f million" %setupCost)
print("BAU Net Present Value = £%.2f million" %BAUNPV)
print("Value of option = £%.2f million" %European_option_value)
print("Value of project = £%.2f million" %European_project_value)

Setup Cost = £720.00 million
BAU Net Present Value = £-41.62 million
Value of option = £434.48 million
Value of project = £-285.52 million


In [ ]:
# Below here is for running the american option
#################################################################################

In [172]:
# ######################
# calculates out option values for different failure situations and stores in multiTreeOption array

multiTreeOption = []
steps = 31
locVal = np.arange(1, assetLife-1, (assetLife-2) / steps )

for n in range(0,len(locVal)): 
    
    averageOptionValue=[]
    
    for _ in range(0,50):    
        tree,optionTree,AoptionTree = InitCalc(TurbineDeathSimulation(scale = 1,loc = locVal[n]) ,steps = 31 , impCostGearbox = impCostGearbox, abdValue = abdValue)        
        averageOptionValue.append(AoptionTree[0,0]-tree[0,0])
         
    multiTreeOption.append( np.mean(averageOptionValue) )
        
multiTreeOption = np.asarray(multiTreeOption)

# ######################
# Estimating mean of standard deviation of logrithmic returns of multiTreeOption

def substitute(multiTreeOption):
    return multiTreeOption
metaSigma = SigmaCalculator(substitute , (multiTreeOption))


print("mean of logrithmic returns standard deviations is: " + str(metaSigma))



# ######################
# Final tree seeding 
    
delta = assetLife / steps 
u = np.exp(metaSigma * np.sqrt(delta)) 
d = 1/u
p = ( np.exp(riskFreeRate * delta) - d ) / (u - d)

finalTree = np.zeros((steps, steps))

if multiTreeOption[0] < multiTreeOption[-1]:  # When seeding largest values must be first as that is how BT works
    finalTree[:,-1] = np.fliplr([multiTreeOption])
else:
    finalTree[:,-1] = multiTreeOption
finalTree[finalTree == 0] = None

#back propagation algorithem
for i in range(steps-1, 0 ,-1):
    for j in range(0,steps-1):
        
        case1 = ( p*finalTree[j , i] + (1-p)*finalTree[j+1 , i] )           
        finalTree[j , i-1] = case1
            
            
pd.DataFrame(finalTree)

mean of logrithmic returns standard deviations is: 0.026068590473253925


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,480.70418,480.578553,480.466092,480.366390,480.279031,480.203588,480.139632,480.086726,480.044434,480.012319,...,480.276925,480.343956,480.417222,480.496811,480.582899,480.675761,480.775780,480.883458,480.999430,481.124472
1,NaN,483.833941,483.380305,482.949978,482.542796,482.158541,481.796944,481.457677,481.140354,480.844524,...,478.701285,478.606977,478.518677,478.434416,478.352083,478.269410,478.183972,478.093173,477.994243,477.884226
2,NaN,NaN,495.135462,494.101095,493.094178,492.115797,491.167069,490.249142,489.363192,488.510428,...,481.348057,481.050776,480.806830,480.617875,480.485598,480.411710,480.397945,480.446064,480.557848,480.735100
3,NaN,NaN,NaN,520.904810,519.186579,517.468730,515.751600,514.035546,512.320953,510.608229,...,490.391282,488.754245,487.128260,485.514281,483.913310,482.326393,480.754628,479.199158,477.661176,476.141922
4,NaN,NaN,NaN,NaN,563.711366,561.983611,560.247875,558.503916,556.751480,554.990310,...,533.074847,531.175021,529.262668,527.337572,525.399541,523.448405,521.484027,519.506300,517.515161,515.510589
5,NaN,NaN,NaN,NaN,NaN,606.755200,605.226259,603.695420,602.162623,600.627791,...,581.986773,580.405515,578.817774,577.222871,575.620053,574.008493,572.387285,570.755436,569.111860,567.455373
6,NaN,NaN,NaN,NaN,NaN,NaN,644.845946,643.364288,641.882216,640.400150,...,622.810182,621.380917,619.961139,618.551857,617.154134,615.769088,614.397899,613.041805,611.702108,610.380178
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,681.758737,680.287370,678.805180,...,660.068041,658.417684,656.752054,655.070798,653.373562,651.659996,649.929750,648.182484,646.417867,644.635576
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,718.415131,717.213392,...,702.447387,701.183651,699.913779,698.637448,697.354297,696.063924,694.765877,693.459651,692.144684,690.820345
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,748.354246,...,735.043200,733.931028,732.820179,731.711219,730.604780,729.501571,728.402377,727.308077,726.219643,725.138156


In [161]:
Am_option_value = finalTree[0,0]
setupCost = findSetupCost(ratedCap, numberOfTurbines, costPerMW)
BAUNPV = tree[0,0]-setupCost
Am_project_value = Am_option_value - setupCost

print("Setup Cost = £%.2f million" %setupCost)
print("BAU Net Present Value = £%.2f million" %BAUNPV)
print("Value of option = £%.2f million" %Am_option_value)
print("Value of project = £%.2f million" %Am_project_value)

720.0
-312.1467658949022
434.48026097586785
-285.51973902413215
